## Installation sur Google Colab

Le package Transformers n&#39;est pas installé par défaut sur Google Colab. Alors installons-le avec pip :


In [1]:
!pip install transformers[sentencepiece]

     |████████████████████████████████| 2.5MB 25.9MB/s 
     |████████████████████████████████| 3.3MB 36.5MB/s 
     |████████████████████████████████| 901kB 32.8MB/s 
     |████████████████████████████████| 1.1MB 24.4MB/s 


## Analyse des sentiments en anglais

Dans cet article, nous utiliserons l&#39;interface de pipeline de haut niveau, ce qui rend extrêmement facile l&#39;utilisation de modèles de transformateurs pré-entraînés.

Fondamentalement, nous avons juste besoin de dire au pipeline ce que nous voulons faire, et éventuellement de lui dire quel modèle utiliser pour cette tâche.

Ici, nous allons faire une analyse des sentiments en anglais, nous sélectionnons donc la tâche `sentiment-analysis` et le modèle par défaut :


In [3]:
from transformers import pipeline
classifier = pipeline("sentiment-analysis")

Le pipeline est prêt, et nous pouvons maintenant l&#39;utiliser :


In [3]:
classifier(["this is a great tutorial, thank you", 
            "your content just sucks"])

[{'label': 'POSITIVE', 'score': 0.9998582601547241},
 {'label': 'NEGATIVE', 'score': 0.9971919059753418}]

Nous avons envoyé deux phrases par le pipeline. Le premier devrait être positif et le second négatif avec un niveau de confiance très élevé.

Ça a l&#39;air bien!

Voyons maintenant ce qui se passe si nous envoyons des phrases en français :


In [4]:
classifier(["Ton tuto est vraiment bien", 
            "il est complètement nul"])

[{'label': 'POSITIVE', 'score': 0.7650704979896545},
 {'label': 'POSITIVE', 'score': 0.8282670974731445}]

Cette fois, le classement ne fonctionne pas...

En effet, la deuxième phrase, qui signifie « ce tuto est de la merde complète », est classée comme positive.

Ce n&#39;est pas une surprise : le modèle par défaut pour la tâche d&#39;analyse des sentiments a été entraîné sur du texte anglais, il ne comprend donc pas le français.


### Analyse des sentiments en néerlandais, allemand, français, espagnol et italien

Alors, que pouvez-vous faire si vous souhaitez travailler avec du texte dans une autre langue, disons le français ?

Il vous suffit de rechercher sur le hub un [modèle de classification français](https://huggingface.co/models?filter=fr&amp;pipeline_tag=text-classification&amp;sort=downloads).

Plusieurs modèles sont disponibles, et j&#39;ai décidé de sélectionner [nlptown/bert-base-multilingual-uncased-sentiment](https://huggingface.co/nlptown/bert-base-multilingual-uncased-sentiment).

Nous pouvons spécifier ce modèle comme celui à utiliser lorsque nous créons notre pipeline `sentiment-analysis` :


In [5]:
multilang_classifier = pipeline("sentiment-analysis", 
                                model="nlptown/bert-base-multilingual-uncased-sentiment")

In [6]:
multilang_classifier(["Ton tuto est vraiment bien", 
                      "il est complètement nul"])

[{'label': '5 stars', 'score': 0.5787978172302246},
 {'label': '1 star', 'score': 0.9223358035087585}]

Et ça a marché ! La deuxième phrase est correctement classée comme très négative.

Vous vous demandez peut-être si la confiance pour la première phrase est plus faible. Je suis presque sûr que c&#39;est parce que cette phrase obtient également un score élevé de « 4 étoiles ».

Essayons maintenant avec une critique réelle pour un restaurant près de chez moi :


In [8]:
import pprint
sentence="Contente de pouvoir retourner au restaurant... Quelle déception... L accueil peu chaleureux... Un plat du jour plus disponible à 12h45...rien à me proposer à la place... Une pizza pas assez cuite et pour finir une glace pleine de glaçons... Et au gout très fade... Je pensais que les serveuses seraient plus aimable à l idée de retrouver leur clientèle.. Dommage"
pprint.pprint(sentence)

('Contente de pouvoir retourner au restaurant... Quelle déception... L accueil '
 'peu chaleureux... Un plat du jour plus disponible à 12h45...rien à me '
 'proposer à la place... Une pizza pas assez cuite et pour finir une glace '
 'pleine de glaçons... Et au gout très fade... Je pensais que les serveuses '
 'seraient plus aimable à l idée de retrouver leur clientèle.. Dommage')


In [9]:
multilang_classifier([sentence])

[{'label': '2 stars', 'score': 0.5843755602836609}]

2 étoiles ! sur Google Review, cet avis a 1 étoile. Pas une mauvaise prédiction.


## Traduction anglais-français

Essayons de faire un peu de traduction, de l&#39;anglais vers le français.

Encore une fois, nous recherchons le hub, et nous nous retrouvons avec ce pipeline :


In [10]:
en_to_fr = pipeline("translation_en_to_fr", 
                    model="Helsinki-NLP/opus-mt-en-fr")

In [12]:
en_to_fr("your tutorial is really good")

[{'translation_text': 'votre tutoriel est vraiment bon'}]

Cela fonctionne bien. Traduisons dans l&#39;autre sens. Pour cela, nous devons changer la tâche et le modèle :


In [13]:
fr_to_en = pipeline("translation_fr_to_en", 
                    model="Helsinki-NLP/opus-mt-fr-en")

In [14]:
fr_to_en("ton tutoriel est super")

[{'translation_text': 'Your tutorial is great.'}]

Excellente traduction !


## Classement zéro tir en français

De nos jours, de très grands modèles d&#39;apprentissage en profondeur sont entraînés sur de très grands ensembles de données collectés sur Internet.

Ces modèles en savent déjà beaucoup, ils n&#39;ont donc pas besoin d&#39;en apprendre beaucoup plus.

En règle générale, il est possible d&#39;affiner ces modèles pour un cas d&#39;utilisation spécifique comme la classification de texte avec un très petit jeu de données spécifique supplémentaire. C&#39;est ce qu&#39;on appelle l&#39;apprentissage en quelques coups.

Et parfois, nous pouvons même faire du zero shot learning : des tâches spécifiques peuvent être effectuées sans aucune formation spécifique. C&#39;est ce que nous allons faire maintenant.

Nous recherchons dans le hub un modèle de classification zéro tir français, et nous créons ce pipeline :


In [15]:
classifier = pipeline("zero-shot-classification", 
                      model="BaptisteDoyen/camembert-base-xlni")

Dans l&#39;exemple ci-dessous, je propose une séquence pour classer en catégories, et je précise également les catégories.

Il est important de noter que le modèle **n&#39;a pas été formé avec ces catégories**, vous pouvez les modifier à volonté !


In [16]:
sequence = "Colin est en train d'écrire un article au sujet du traitement du langage naturel"
candidate_labels = ["science","politique","education", "news"]
classifier(sequence, candidate_labels)     

{'labels': ['science', 'news', 'education', 'politique'],
 'scores': [0.4613836407661438,
  0.20861364901065826,
  0.20573210716247559,
  0.12427058815956116],
 'sequence': "Colin est en train d'écrire un article au sujet du traitement du langage naturel"}

Les probabilités prédites sont en fait assez solides ! Cette phrase concerne en effet la science, l&#39;actualité et l&#39;éducation. Et pas du tout lié à la politique.

Mais si nous essayons celui-ci :


In [21]:
sequence = "Laurent Wauquiez reconduit à la tête de la région Rhône-Alpes-Auvergne à la suite du deuxième tour des élections."
candidate_labels = ["politique", "musique"]
classifier(sequence, candidate_labels)   

{'labels': ['politique', 'musique'],
 'scores': [0.6573010087013245, 0.34269899129867554],
 'sequence': 'Laurent Wauquiez reconduit à la tête de la région Rhône-Alpes-Auvergne à la suite du deuxième tour des élections.'}

Ça marche!

N&#39;hésitez pas à essayer d&#39;autres phrases et d&#39;autres catégories. Vous pouvez également changer de modèle si vous souhaitez effectuer une classification zéro tir en anglais ou dans une autre langue.


## Résumé en français

Résumer du texte est une application intéressante des transformateurs.

Ici, nous utilisons un modèle entraîné sur un jeu de données obtenu par grattage [https://actu.orange.fr/](https://actu.orange.fr/), à nouveau trouvé sur le hub Hugging Face :



In [22]:
summarizer = pipeline("summarization", 
                       model="moussaKam/barthez-orangesum-title")

Reprenons les deux premiers paragraphes d&#39;un article sur le Covid-19 lu dans Le Monde :


In [26]:
import pprint
sentence = "La pandémie ne marque pas le pas. Le variant Delta poursuit son essor planétaire au grand dam de pays impatients de retrouver une vie normale. La pandémie a fait près de quatre millions de morts dans le monde depuis que le bureau de l’Organisation mondiale de la santé (OMS) en Chine a fait état de l’apparition de la maladie fin décembre 2019, selon un bilan établi par l’Agence France-Presse (AFP) à partir de sources officielles, lundi à 12 heures. Les Etats-Unis sont le pays le plus touché tant en nombre de morts (603 967) que de cas. Le Brésil, qui compte 513 474 morts, est suivi par l’Inde (396 730), le Mexique (232 564) et le Pérou (191 899), le pays qui déplore le plus de morts par rapport à sa population. Ces chiffres, qui reposent sur les bilans quotidiens des autorités nationales de santé, sont globalement sous-évalués. L’Organisation mondiale de la santé (OMS) estime que le bilan de la pandémie pourrait être deux à trois fois plus élevé que celui officiellement calculé."
pprint.pprint(sentence)

('La pandémie ne marque pas le pas. Le variant Delta poursuit son essor '
 'planétaire au grand dam de pays impatients de retrouver une vie normale. La '
 'pandémie a fait près de quatre millions de morts dans le monde depuis que le '
 'bureau de l’Organisation mondiale de la santé (OMS) en Chine a fait état de '
 'l’apparition de la maladie fin décembre 2019, selon un bilan établi par '
 'l’Agence France-Presse (AFP) à partir de sources officielles, lundi à 12 '
 'heures. Les Etats-Unis sont le pays le plus touché tant en nombre de morts '
 '(603 967) que de cas. Le Brésil, qui compte 513 474 morts, est suivi par '
 'l’Inde (396 730), le Mexique (232 564) et le Pérou (191 899), le pays qui '
 'déplore le plus de morts par rapport à sa population. Ces chiffres, qui '
 'reposent sur les bilans quotidiens des autorités nationales de santé, sont '
 'globalement sous-évalués. L’Organisation mondiale de la santé (OMS) estime '
 'que le bilan de la pandémie pourrait être deux à trois fois pl

In [30]:
summarizer(sentence, max_length=80)

[{'summary_text': 'Coronavirus : près de 4 millions de morts dans le monde'}]

Lapidaire, mais pas mal !


## Reconnaissance de l&#39;entité nommée en français

La reconnaissance d&#39;entités nommées peut servir de base à de nombreuses applications intéressantes !

Par exemple, on pourrait analyser des rapports financiers à la recherche de dates, de prix, de noms de sociétés.

Voyons comment faire cela.

Ici, nous utilisons un équivalent français de BERT, appelé CamemBERT, affiné pour NER :


In [4]:
ner = pipeline("token-classification", model="Jean-Baptiste/camembert-ner")

In [7]:
nes = ner("Colin est parti à Saint-André acheter de la mozzarella")
pprint.pprint(nes)

[{'end': 5,
  'entity': 'PER',
  'index': 1,
  'score': 0.94243556,
  'start': 0,
  'word': '▁Colin'},
 {'end': 23,
  'entity': 'LOC',
  'index': 5,
  'score': 0.99605554,
  'start': 17,
  'word': '▁Saint'},
 {'end': 24,
  'entity': 'LOC',
  'index': 6,
  'score': 0.9967083,
  'start': 23,
  'word': '-'},
 {'end': 29,
  'entity': 'LOC',
  'index': 7,
  'score': 0.99609375,
  'start': 24,
  'word': 'André'}]


Nous devons faire un peu de post-traitement pour agréger les entités nommées du même type.

Voici un algorithme simple pour le faire (il peut certainement être amélioré !)


In [10]:
cur = None
agg = []
for ne in nes: 
  entity=ne['entity']
  if entity != cur: 
    if cur is None: 
      cur = entity
    if agg: 
      print(cur, ner.tokenizer.convert_tokens_to_string(agg))
      agg = []
      cur = entity
  agg.append(ne['word'])
print(cur, ner.tokenizer.convert_tokens_to_string(agg))


PER Colin
LOC Saint-André


Nous avons trouvé deux entités nommées :

* PERSONNE : Colin
* LIEU : Saint-André
